```text
SPDX-FileCopyrightText: 2023 Google LLC
SPDX-License-Identifier: Apache-2.0
```


# EuroPython 2023 Cloud Challenge

## Build your Zen-erator with Vertex AI

The **Zen of Python** provides very useful guiding principles:

- Beautiful is better than ugly.
- Explicit is better than implicit.
- Simple is better than complex.
- Complex is better than complicated.
- Flat is better than nested.
- …

Let's generate your own "Zen of" using our latest Google Cloud LLM model, with the [Vertex AI PaLM API](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/generative-ai-studio).

---

## 💻 Google Cloud

- To complete this Cloud Challenge, you'll need a Google Cloud project.
- Redeem a 1 USD credit from [https://trygcp.dev/europython-2023](https://trygcp.dev/europython-2023), which will more than cover any potential cost.
- If you're already a Google Cloud user, create a new project and link it to the redeemed credit billing account.
- If you're new to Google Cloud,
  - This will get you started without a credit card.
  - Sign in with a Google account (your gmail account will do).
  - Follow the steps.
  - A "My First Project" will be created for you.
  - Copy your Project ID from the URL (`&project=YOUR-PROJECT-ID`).

---

## ✔️ Setup


In [ ]:
# @title 📦️ Make sure you have the right packages (may restart) {display-mode: "form"}

import sys
from importlib import metadata

from IPython.core.getipython import get_ipython
from packaging import version

# Services needed for this lab (with minimum version)
# Note: This assumes that earlier versions are non-breaking
GOOGLE_CLOUD_SERVICES = [
    ("aiplatform", "1.25.0"),
]
APIS = [f"{service}.googleapis.com" for service, _ in GOOGLE_CLOUD_SERVICES]

# Check runtime
running_in_colab = "google.colab" in sys.modules
assert running_in_colab, "❌ The notebook was only designed to run in Colab"
print("✔️ Running in Colab")

# Check packages
packages = []
for service, min_version_str in GOOGLE_CLOUD_SERVICES:
    package = f"google-cloud-{service}"
    min_version = version.parse(min_version_str)
    try:
        lib = f"google.cloud.{service}"
        lib_version = version.parse(metadata.version(lib))
        if min_version <= lib_version:
            print(f"✔️ {lib}=={lib_version!s}")
            continue
        packages.append(package)
        print(f"📦️ {package} to be updated…")
    except metadata.PackageNotFoundError:
        packages.append(package)
        print(f"📦️ {package} to be installed…")

if packages:
    # Install and restart
    requirements = " ".join(packages)
    %pip install --upgrade $requirements --quiet
    if instance := get_ipython():
        instance.kernel.do_shutdown(True)
    raise RuntimeWarning("🔄 Restarting… (run the cell again)")

In [ ]:
# @title ⚙️ Enter your Google Cloud project ID {display-mode: "form"}

PROJECT_ID = ""  # @param {type:"string"}

assert PROJECT_ID, "❌ Please enter your project ID"

print(f'✔️ PROJECT_ID: "{PROJECT_ID}"')

In [ ]:
# @title 🔑 Authenticate (authorize your Google account to run this codelab) {display-mode: "form"}
from google.colab import auth

auth.authenticate_user(project_id=PROJECT_ID)
print(f"✔️ Authenticated")

In [ ]:
# @title 🔓 Make sure the Vertex AI API is enabled {display-mode: "form"}
res = !gcloud services list --enabled --format "value(config.name)"

apis_to_enable = ""
for api in APIS:
    if api in res:
        print(f'✔️ API "{api}" is enabled')
    else:
        apis_to_enable += f"{api} "

if apis_to_enable:
    print(f'🔓 Enabling API "{apis_to_enable}"…')
    !gcloud services enable $api
elif not APIS:
    print(f"✔️ No specific API needed")

In [ ]:
# @title 🛠️ Define helper functions {display-mode: "form"}

import base64
import json

import pandas as pd
import requests
import vertexai
from IPython.display import display, display_markdown
from vertexai.language_models import TextGenerationModel, TextGenerationResponse


def predict(
    prompt: str,
    temperature: float = 0.5,
    max_output_tokens: int = 512,
    top_p: float = 0.8,
    top_k: int = 40,
) -> TextGenerationResponse:
    """
    prompt: Question to ask the model
    temperature: Ratio [0..1] of randomness in token selection
    max_output_tokens: Max length of the output text in tokens
    top_p: Tokens are selected from most probable to least until the sum of their probabilities equals the top_p value
    top_k: Number of highest probability vocabulary tokens to keep for top-k-filtering
    """
    LOCATION = "us-central1"

    vertexai.init(project=PROJECT_ID, location=LOCATION)
    model = TextGenerationModel.from_pretrained("text-bison")
    PARAMETERS = {
        "temperature": temperature,
        "max_output_tokens": max_output_tokens,
        "top_p": top_p,
        "top_k": top_k,
    }

    return model.predict(prompt, **PARAMETERS)


def get_sentences(response: TextGenerationResponse) -> list[str]:
    try:
        sentences = json.loads(response.text)
        if isinstance(sentences, list):
            return sentences
        print("Error: Could not decode the answer as a string list.")
    except json.JSONDecodeError:
        print(
            "Error: Could not decode the answer as JSON. If the generated answer was too long/truncated, retry or increase the parameter max_output_tokens in the request."
        )
    return []


def emoji_digits_for_integer(i: int) -> str:
    emojis = ""
    while True:
        i, digit = divmod(i, 10)
        emoji = "0️⃣1️⃣2️⃣3️⃣4️⃣5️⃣6️⃣7️⃣8️⃣9️⃣"[3 * digit : 3 * (digit + 1)]
        emojis = f"{emoji}{emojis}"
        if i == 0:
            return emoji


def display_zenof(topic: str, sentences: list[str]):
    data = (
        (emoji_digits_for_integer(number), sentence)
        for number, sentence in enumerate(sentences, 1)
    )
    df = pd.DataFrame(data=data)
    styler = df.style.set_properties(subset=[1], **{"text-align": "left"})
    styler.hide(axis=0).hide(axis=1)

    display_markdown(f'# Here\'s your Zen of "{topic}"', raw=True)
    display(styler)


def register(
    status: str,
    name: str,
    badge_id: str,
    eligible: bool,
    topic: str,
    sentences: list[str],
):
    url = "https://ep23.lolo.dev/cloud_challenge_register"
    status = status.encode("utf-16", "surrogatepass").decode("utf-16")
    name = name.encode("utf-16", "surrogatepass").decode("utf-16")
    zenof = dict(topic=topic, sentences=sentences)
    zenof_b64 = base64.b64encode(json.dumps(zenof).encode()).decode()
    context_slist = !gcloud config list --format json
    context = json.loads("".join(context_slist))
    context_b64 = base64.b64encode(json.dumps(context).encode()).decode()
    data = dict(
        status=status,
        name=name,
        badge_id=badge_id,
        eligible=eligible,
        zenof_b64=zenof_b64,
        context_b64=context_b64,
    )

    response = requests.post(url, data)
    if response.status_code == 200:
        message = response.text
    else:
        message = f"❌ {response.status_code}: {response.text}"
    print(message)


print(f"✔️ Defined helper functions")

---
## 🐍 Build your Zen-erator

In [ ]:
# @title Enter your favorite topic for your "Zen of". {display-mode: "form"}

TOPIC = ""  # @param {type:"string"}

assert TOPIC, "❌ Please enter your topic"

print(f'✔️ TOPIC: "{TOPIC}"')

We have crafted a possible prompt for you.

In [ ]:
prompt = f"""
For the following topic, return 5 sentences as string list in JSON.
Each sentence must be a positive principle inspired by the Zen philosophy, as short as possible.
TOPIC: {TOPIC}
JSON:
"""

print(f'Here is a possible prompt for your Zen-erator about "{TOPIC}":\n↓{prompt}↑')

This is the helper function we previously defined to call the PaLM API:

```python
def predict(
    prompt: str,
    temperature: float = 0.5,
    max_output_tokens: int = 512,
    top_p: float = 0.8,
    top_k: int = 40,
) -> TextGenerationResponse:
    vertexai.init(project=PROJECT_ID, location="us-central1")
    model = TextGenerationModel.from_pretrained("text-bison")
    PARAMETERS = {
        "temperature": temperature,
        "max_output_tokens": max_output_tokens,
        "top_p": top_p,
        "top_k": top_k,
    }
    return model.predict(prompt, **PARAMETERS)
```

Call the PaLM API to generate your "Zen of".

In [ ]:
response = predict(prompt, temperature=0.6)

sentences = get_sentences(response)

display_zenof(TOPIC, sentences)

You can try a few times until you're happy with your generated "Zen of".

> Notes:
> - If you want the model to be as creative as possible (introduce more randomness), set the temperature parameter to `1.0`.
> - You can also try to finetune and improve the prompt. Let us know if you manage to get better results with a different prompt.

---

## 🏆 Register for the daily raffles


In [ ]:
# @markdown Select (or enter) 1 emoji that represents your mood.
STATUS = "🚀"  # @param ["🚀","👍","🥰","🎉","🔴","🟠","🟡","🟢","🔵"] {allow-input: true}

# @markdown Enter your full name (as printed on your EuroPython badge).
NAME = "Jane Doe"  # @param {type:"string"}

# @markdown Enter your badge ID (bottom-left corner of your badge).
BADGE_ID = "#ABCDE-N"  # @param {type:"string"}

# @markdown Are you eligible to participate? (set to False if you're a Googler)
ELIGIBLE = True  # @param {type:"boolean"}

register(STATUS, NAME, BADGE_ID, ELIGIBLE, TOPIC, sentences)

If you get the message `👍 You're registered!`, then you're all set. You have a chance to win a prize during one of the 3 daily raffles.

---

## 🎉 Congratulations

- ✔️ You built a Zen-erator powered by the Vertex AI PaLM API.
- ✔️ You registered for the daily raffles!

Don't forget to come back to our booth Wednesday/Thursday/Friday for each daily raffle:

- ⏰ See you at **15:15**
- ☕ That's right in the **middle of the afternoon break**

> If your name is drawn but you're not there, we'll have to relaunch the wheel to give the prize to another winner.

Feel free to come to our booth anytime to chat about your "Zen of", Google Cloud, Python, or anything.

We wish you the best EuroPython!